# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: *** @Rebecamoreno_***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Rebecamoreno_

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Nutella'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [121]:
dft = pd.read_excel('./{0}.xlsx'.format(produto),sheet='Treinamento')

dftsim = dft[(dft.Relevante=="Sim")]
dftnao = dft[(dft.Relevante=="Não")]

In [8]:
lendo = pd.ExcelFile('./{0}.xlsx'.format(produto))
dfteste = lendo.parse("Teste")

In [22]:
# A, S e N são dicionários com as frequências de cada uma das palavras separadas (splitadas).
A = {}
S = {}
N = {}
todas = 0
c=0
#para cada post (msg)
for msg in dft["Treinamento"]:
    #para cada palavra dentro do post (i)
    #msg = msg.split()
    for i in msg.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        if not i in A:
            A[i]=0
        A[i]+=1
        todas+=1
        if A[i]>=c:
            c=A[i]
            string = i

print('Como curiosidade, a palavra que aparece com mais frequência em toda essa análise é a palavra "{0}", com uma frenquência de {1} vezes.\n'.format(string,c))
todas_em_sim = 0
p = 0
#para cada post (msg)
for msg in dftsim["Treinamento"]:
    #para cada palavra dentro do post (i)
    #msg = msg.split()
    for i in msg.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        if not i in S:
            S[i]=0
        S[i]+=1
        todas_em_sim+=1
        if S[i]>=p:
            p=S[i]
            strings = i
            
print('Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de RELEVANTES é a palavra "{0}", com uma frenquência de {1} vezes.\n'.format(strings,p))
todas_em_nao = 0
f = 0
#para cada post (msg)
for msg in dftnao["Treinamento"]:
    #para cada palavra dentro do post (i)
    #msg = msg.split()
    for i in msg.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        if not i in N:
            N[i]=0
        N[i]+=1
        todas_em_nao+=1
        if N[i]>=f:
            f=N[i]
            stringn = i
            
print('Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de NÃO RELEVANTES é a palavra "{0}", com uma frenquência de {1} vezes.\n'.format(stringn,f))
print('A quantidade TOTAL de palavras no UNIVERSO é {0}.'.format(todas))
print('A quantidade TOTAL de palavras dentro de RELEVANTES é {0}.'.format(todas_em_sim))
print('A quantidade TOTAL de palavras dentro de NÃO RELEVANTES é {0}.\n'.format(todas_em_nao))
print('A quantidade de palavras DIFERENTES no UNIVERSO é {0}.'.format(len(A)))
print('A quantidade de palavras DIFERENTES dentro de RELEVANTES é {0}.'.format(len(S)))
print('A quantidade de palavras DIFERENTES dentro de NÃO RELEVANTES é {0}.'.format(len(N)))

Como curiosidade, a palavra que aparece com mais frequência em toda essa análise é a palavra "nutella", com uma frenquência de 219 vezes.

Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de RELEVANTES é a palavra "nutella", com uma frenquência de 135 vezes.

Como curiosidade, a palavra que aparece com mais frequência entre todas as palavras de NÃO RELEVANTES é a palavra "nutella", com uma frenquência de 84 vezes.

A quantidade TOTAL de palavras no UNIVERSO é 3774.
A quantidade TOTAL de palavras dentro de RELEVANTES é 1750.
A quantidade TOTAL de palavras dentro de NÃO RELEVANTES é 2024.

A quantidade de palavras DIFERENTES no UNIVERSO é 1370.
A quantidade de palavras DIFERENTES dentro de RELEVANTES é 636.
A quantidade de palavras DIFERENTES dentro de NÃO RELEVANTES é 917.


In [125]:
#Calculando a probabilidade de cada palavra
#Conta: P(sim|palavra)=(P(palavra|sim)+1) / P(palavras totais|sim)+P(palavras totais sem repetição)
bs = todas_em_sim
bn = todas_em_nao
c = len(A)
PS = {}
PN = {}
TSim = dftsim["Treinamento"].count()
TNao = dftnao["Treinamento"].count()
TTotal = dft["Treinamento"].count()

#PTeste = {}

for msg2 in dfteste['Teste']:
    #Para sim:
    Ptweet = (TSim/TTotal)
    for i in msg2.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        
        if i in S:
            PS[i] = (S[i]+1)/(bs+c)
            
        else:
            PS[i] = 1/(bs+c)
            
        Ptweet = Ptweet*PS[i]
        print(Ptweet)
        print(i)
        
    #Para não:   
    for i in msg2.split():
        #analisando cada palavra ja separada (for e if)
        for caracter in range(0,len(i)):
            i = i.strip(",").strip("'").strip('"').strip('#').strip(':').strip(';').strip('!').strip('(').strip(')').strip('/').strip('\n').strip('.').strip('\\').strip('-').strip('$').strip('%').strip('|').strip('=').strip('*').strip('ˆ').strip('&').strip('+').strip('?')
        #contando sua frenquencia e adicionando em A
        
        if i in N:
            PN[i] = (N[i]+1)/(bn+c)
            
        else:
            PN[i] = 1/(bn+c)
        
#print(PN)
#print(PS["nutella"])

0.00530769230769
rt
1.70118343195e-06
@br00talgirl
5.45251099985e-10
espero
4.89327910243e-12
um
3.13671737335e-15
dia
4.02143252994e-18
ser
2.57784136534e-21
feliz
1.65246241368e-24
igual
3.177812334e-27
esse
1.01852959423e-30
rato
5.87613227442e-33
no
1.88337572898e-36
centro
5.37244999613e-38
de
1.72193910132e-41
barueri
5.51903558116e-45
https://t.co/rpuuf7moe9
0.000147435897436
adicionei
1.3231426693e-06
um
4.24084188877e-10
vídeo
2.17479071219e-12
a
1.18498211882e-14
uma
3.79801961161e-18
playlist
1.21731397808e-21
@youtube
3.90164736564e-25
https://t.co/gkczigpl9e
1.11296992161e-26
de
3.56721128721e-30
1
1.14333695103e-33
milhão
2.56517905679e-36

4.93303664768e-39
bolo
4.90141461789e-41
ninho
8.32612098551e-43
com
3.62933478856e-44
nutella
1.16324832967e-47
vulcão
3.72836003098e-51
massa
1.19498718942e-54
sabor
0.000147435897436
@pedronic_
4.7255095332e-08
@sleepool
1.51458638885e-11
@gabriel0h
4.854443554e-15
@prdww
1.55591139551e-18
lucas
4.98689549843e-22
neto
7.35247413231e

1.09050219997e-09
tanto
1.04855980766e-12
doce
3.3607686143e-15
pra
1.07716942766e-18
harmonizar
1.58813441258e-20
e
1.01803487986e-23
vcs
3.26293230724e-27
insistem
1.04581163693e-30
nessa
3.35196037479e-34
combinação
9.56168183835e-36
de
1.04197814905e-37
leite
1.03529880194e-39
ninho
1.75868065714e-41
com
7.66604389011e-43
nutella
7.37119604819e-46
tão
9.45025134383e-49
enjoativo
0.00250641025641
o
2.73134451019e-05
que
8.75430932753e-09
faria
2.52528153679e-11
se
8.09385107945e-15
acordasse
2.30882290407e-16
de
7.40007341049e-20
madrugada
1.09103646437e-21
e
5.94474996611e-24
o
1.90536857888e-27
chucky
6.10695057333e-31
es
1.95735595299e-34
—
6.27357677241e-38
ia
4.02152357206e-41
falar
4.38242953365e-43
que
3.37109964127e-45
eu
3.24144196276e-48
amo
1.41293624018e-49
nutella
2.0831752259e-51
e
6.67684367277e-55
essas
2.14001399768e-58
concerteza
6.85901922334e-62
seri
2.19840359722e-65
https://t.co/esqhbyohfm
0.00530769230769
rt
1.70118343195e-06
@mrcioluiz91
5.45251099985e-10
@ko

acabaram
4.0081025699e-32
agora
3.9824096047e-34
é
1.78697866878e-36
só
5.72749573326e-40
bicha
2.49660070424e-41
nutella
8.00192533411e-45
diria
1.79530376086e-47
pão
3.04971472197e-49
com
9.77472667298e-53
ovo
0.000147435897436
@r10_bottega
4.7255095332e-08
jogador
2.05983748883e-09
nutella
0.000147435897436
@nutella_silva
4.7255095332e-08
arrocha
6.05834555539e-11
tá
1.9417774216e-14
aaaiii
0.000147435897436
@welington_wc
1.89020381328e-07
coisa
5.39192754429e-09
de
1.72818190522e-12
velho
3.87733119761e-15
isso
4.97093743284e-18
aí
3.18649835438e-21
vcs
1.02131357512e-24
são
6.54688189182e-28
muito
2.85376902977e-29
nutella
9.14669560822e-33
usam
1.46581660388e-35
até
4.69813014065e-39
luzinha
0.000147435897436
perfil
4.7255095332e-08
visualizado
1.36312774996e-10
por
4.3689991986e-14
8
4.20096076788e-17
pessoas
1.34646178458e-20
https://t.co/myz0u6bo7s
0.00530769230769
rt
1.70118343195e-06
@joaqulnvoltou
4.90725989986e-09
se
1.57283971149e-12
chegar
5.04115292146e-16
nesse
1.61575

7.37580038092e-39
recheio
2.36403858363e-42
era
6.74357160073e-44
de
2.93950556955e-45
nutella
9.4214922101e-49
tirou
3.0197090417e-52
ondaa
0.00530769230769
rt
1.70118343195e-06
@isasantana2403
7.63351539979e-09
só
2.20197559609e-11
queria
1.97613194521e-13
um
1.20341368458e-15
morango
2.04426042574e-17
com
8.91087877885e-19
nutella
2.28484071252e-21
agora
0.000147435897436
https://t.co/56amzinurr
2.83530571992e-07
quero
1.81750366662e-10
muito
1.63109303414e-12
um
5.75064851781e-15
pote
1.64040935284e-16
de
7.15050230724e-18
nutella
0.00530769230769
rt
1.70118343195e-06
@alexmapelli0
2.72625549992e-09
já
1.74759967944e-12
fui
5.60128102384e-16
10/10
9.51499661101e-18
com
3.04967840097e-21
quem
2.24816035969e-23
não
7.20564217848e-27
merecia
9.23800279292e-30
nem
2.96089833107e-33
5
9.49005875342e-37
também
1.52084274894e-39
já
9.74899198042e-43
fui
3.1246769168e-46
2
5.30794476251e-48
com
1.70126434696e-51
quem
5.45277034282e-55
merecia
1.74768280219e-58
11
8.96247590864e-61
a
1.4362

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
